In [ ]:
# @title **ipSAE_batch: Protein Interface Analysis**
# @markdown ---
# @markdown ### 🧬 **Batch Processing for Protein Structure Predictions**
# @markdown
# @markdown This notebook runs **ipSAE_batch v1.0.0** analysis on protein structure predictions to calculate interaction scores and generate visualizations.
# @markdown
# @markdown **What ipSAE_batch Does:**
# @markdown - Calculates **ipSAE** (interface predicted Structural Alignment Error) scores
# @markdown - Generates **AlphaBridge-style** matrix and ribbon visualizations
# @markdown - Supports multiple backends: AlphaFold3, ColabFold, Boltz2, IntelliFold
# @markdown - Outputs per-contact and per-residue scores
# @markdown
# @markdown **Required Input:**
# @markdown - Structure prediction output folder (ZIP or from Google Drive)
# @markdown - Selection of appropriate backend
# @markdown
# @markdown **Expected Runtime:** 1-5 minutes per job depending on number of models
# @markdown
# @markdown ---
# @markdown
# @markdown 📚 **GitHub:** [JKourelis/ipSAE_batch](https://github.com/JKourelis/ipSAE_batch)
# @markdown
# @markdown 📄 **Citations:**
# @markdown - ipSAE: [Dunbrack 2025](https://doi.org/10.1101/2025.02.10.637595)
# @markdown - AlphaBridge: [Álvarez-Salmoral et al. 2024](https://doi.org/10.1101/2024.10.23.619601)
# @markdown
# @markdown ---

print("╔════════════════════════════════════════════════════════════╗")
print("║         ipSAE_batch v1.0.0 - Interface Analysis           ║")
print("║   Protein Structure Prediction Scoring & Visualization    ║")
print("╚════════════════════════════════════════════════════════════╝")
print("\n📋 Follow the cells in order:")
print("   1. Install ipSAE_batch and dependencies")
print("   2. Connect to Google Drive (optional)")
print("   3. Configure analysis parameters")
print("   4. Select prediction data")
print("   5. Run analysis")
print("   6. Download results\n")
print("🔬 Supported backends:")
print("   • AlphaFold3 (AF3 server or local)")
print("   • ColabFold (AF2 multimer)")
print("   • Boltz2")
print("   • IntelliFold\n")

In [ ]:
# @title **Install ipSAE_batch** { display-mode: "form" }
# @markdown Install ipSAE_batch and all required dependencies.
# @markdown This takes approximately 1-2 minutes.

import subprocess
import sys
import os
from datetime import datetime

print("="*60)
print("ENVIRONMENT SNAPSHOT")
print("="*60)
print(f"⏰ Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Python: {sys.version.split()[0]}")

# Check GPU
try:
    gpu_info = subprocess.run(['nvidia-smi', '--query-gpu=name,driver_version,memory.total', '--format=csv,noheader'],
                              capture_output=True, text=True, timeout=10)
    if gpu_info.returncode == 0 and gpu_info.stdout.strip():
        gpu_name, driver, memory = gpu_info.stdout.strip().split(', ')
        print(f"🎮 GPU: {gpu_name}")
        print(f"   Driver: {driver}")
        print(f"   Memory: {memory}")
    else:
        print("🎮 GPU: None detected (CPU mode)")
except:
    print("🎮 GPU: None detected (CPU mode)")

print("="*60)
print("\n📦 Installing ipSAE_batch...\n")

# Install pycirclize and igraph first (not pre-installed on Colab)
packages = [
    "pycirclize==1.7.1",
    "igraph==0.11.8",
]

for pkg in packages:
    print(f"   Installing {pkg}...")
    result = subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg],
                           capture_output=True, text=True)
    if result.returncode != 0:
        print(f"   ⚠️ Warning: {result.stderr}")

# Install ipSAE_batch from GitHub
print("   Installing ipSAE_batch from GitHub...")
result = subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q",
     "git+https://github.com/JKourelis/ipSAE_batch.git"],
    capture_output=True, text=True
)

if result.returncode != 0:
    print(f"❌ Installation failed: {result.stderr}")
else:
    print("\n" + "="*60)
    print("VERIFICATION")
    print("="*60)

    # Verify installation
    try:
        import ipsae_batch
        print(f"✅ ipSAE_batch version: {ipsae_batch.__version__}")

        import numpy as np
        print(f"✅ numpy version: {np.__version__}")

        import pandas as pd
        print(f"✅ pandas version: {pd.__version__}")

        import scipy
        print(f"✅ scipy version: {scipy.__version__}")

        import matplotlib
        print(f"✅ matplotlib version: {matplotlib.__version__}")

        import pycirclize
        print(f"✅ pycirclize version: {pycirclize.__version__}")

        import igraph
        print(f"✅ igraph version: {igraph.__version__}")

        print("\n🎉 Installation successful! Proceed to the next cell.")

    except ImportError as e:
        print(f"❌ Import error: {e}")
        print("   Please try running this cell again.")

In [ ]:
# @title **Connect to Google Drive** { display-mode: "form" }
# @markdown Mount your Google Drive to access files and save results persistently.
# @markdown
# @markdown **Note:** After mounting, your Drive contents will be at `/content/drive/MyDrive/`

import os
from google.colab import drive

try:
    drive.mount('/content/drive', force_remount=False)
    print("✅ Google Drive connected successfully!")
    print(f"📁 Drive mounted at: /content/drive/MyDrive/")

    # Create ipSAE results directory
    results_dir = "/content/drive/MyDrive/ipSAE_Results"
    os.makedirs(results_dir, exist_ok=True)
    print(f"📂 Results will be saved to: {results_dir}")

    # List top-level folders in Drive
    print("\n" + "="*60)
    print("📂 YOUR GOOGLE DRIVE CONTENTS (top level):")
    print("="*60)
    drive_root = "/content/drive/MyDrive"
    items = sorted(os.listdir(drive_root))
    folders = [f for f in items if os.path.isdir(os.path.join(drive_root, f))][:20]
    files_list = [f for f in items if os.path.isfile(os.path.join(drive_root, f)) and (f.endswith('.zip') or f.endswith('.tar.gz'))][:10]

    print("\n📁 Folders:")
    for folder in folders:
        print(f"   /content/drive/MyDrive/{folder}")
    if len([f for f in items if os.path.isdir(os.path.join(drive_root, f))]) > 20:
        print(f"   ... and more folders")

    if files_list:
        print("\n📦 ZIP/Archive files:")
        for f in files_list:
            print(f"   /content/drive/MyDrive/{f}")

    print("\n💡 Copy the full path above to use in the next cell.")

except Exception as e:
    print(f"⚠️ Google Drive not connected: {e}")
    print("   You can still upload files directly in the next cell.")

In [ ]:
# @title **Configure Analysis Parameters** { display-mode: "form" }
# @markdown Set the parameters for your ipSAE analysis.

# @markdown ---
# @markdown ### Backend Selection
backend = "alphafold3" # @param ["alphafold3", "colabfold", "boltz2", "intellifold"]
# @markdown - **alphafold3**: AF3 server or local AlphaFold3 outputs
# @markdown - **colabfold**: ColabFold/AF2 multimer outputs
# @markdown - **boltz2**: Boltz2 prediction outputs
# @markdown - **intellifold**: IntelliFold server outputs

# @markdown ---
# @markdown ### Analysis Parameters
pae_cutoff = 10.0 # @param {type:"number"}
# @markdown PAE cutoff for ipSAE calculation (default: 10.0)

distance_cutoff = 10.0 # @param {type:"number"}
# @markdown Distance cutoff for interface residues in Ångströms (default: 10.0)

# @markdown ---
# @markdown ### Output Options
generate_png = True # @param {type:"boolean"}
# @markdown Generate PNG graphics (matrix and ribbon plots)

generate_pdf = False # @param {type:"boolean"}
# @markdown Generate PDF report

per_contact_output = True # @param {type:"boolean"}
# @markdown Output per-contact scores CSV

per_residue_output = False # @param {type:"boolean"}
# @markdown Output per-residue scores CSV

# @markdown ---

# Store configuration
config = {
    'backend': backend,
    'pae_cutoff': pae_cutoff,
    'distance_cutoff': distance_cutoff,
    'generate_png': generate_png,
    'generate_pdf': generate_pdf,
    'per_contact_output': per_contact_output,
    'per_residue_output': per_residue_output
}

print("="*60)
print("CONFIGURATION SUMMARY")
print("="*60)
print(f"🔧 Backend: {backend}")
print(f"📊 PAE cutoff: {pae_cutoff}")
print(f"📏 Distance cutoff: {distance_cutoff} Å")
print(f"🖼️  Generate PNG: {generate_png}")
print(f"📄 Generate PDF: {generate_pdf}")
print(f"📋 Per-contact output: {per_contact_output}")
print(f"📋 Per-residue output: {per_residue_output}")
print("="*60)
print("\n✅ Configuration saved. Proceed to select your data.")

In [ ]:
# @title **Select Prediction Data** { display-mode: "form" }
# @markdown Choose how to load your structure prediction data.

# @markdown ---
# @markdown ### Data Source
upload_method = "Google Drive path" # @param ["Google Drive path", "Google Drive ZIP file", "Upload ZIP file"]

# @markdown ---
# @markdown ### Google Drive Path
# @markdown Enter the **full path** to your folder or ZIP file.
# @markdown
# @markdown **Examples:**
# @markdown - `/content/drive/MyDrive/AF3_predictions`
# @markdown - `/content/drive/MyDrive/my_folder/boltz2_results.zip`
drive_path = "/content/drive/MyDrive/" # @param {type:"string"}

# @markdown ---

import os
import zipfile
import shutil
from google.colab import files

input_folder = None

print("="*60)
print("📁 DATA SELECTION")
print("="*60)

def extract_zip(zip_path, extract_to):
    """Extract ZIP file and return the data folder path."""
    if os.path.exists(extract_to):
        shutil.rmtree(extract_to)
    os.makedirs(extract_to)

    print(f"📦 Extracting {os.path.basename(zip_path)}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    # Handle nested folders (ZIP containing single folder)
    contents = os.listdir(extract_to)
    if len(contents) == 1 and os.path.isdir(os.path.join(extract_to, contents[0])):
        return os.path.join(extract_to, contents[0])
    return extract_to

def list_folder_contents(folder_path, max_items=15):
    """List folder contents."""
    print(f"\n📂 Contents of {folder_path}:")
    items = sorted(os.listdir(folder_path))
    for item in items[:max_items]:
        item_path = os.path.join(folder_path, item)
        if os.path.isdir(item_path):
            subcount = len(os.listdir(item_path))
            print(f"   📁 {item}/ ({subcount} items)")
        else:
            size_kb = os.path.getsize(item_path) / 1024
            print(f"   📄 {item} ({size_kb:.1f} KB)")
    if len(items) > max_items:
        print(f"   ... and {len(items) - max_items} more items")

# ============================================================
# OPTION 1: Google Drive folder path
# ============================================================
if upload_method == "Google Drive path":
    path = drive_path.strip()

    if not path:
        print("❌ Please enter a path in the 'drive_path' field above.")
    elif not path.startswith('/content/drive/'):
        print("❌ Path must start with '/content/drive/MyDrive/'")
        print(f"   You entered: {path}")
        print("\n💡 Example: /content/drive/MyDrive/your_folder")
    elif not os.path.exists(path):
        print(f"❌ Path not found: {path}")
        print("\n💡 Possible issues:")
        print("   1. Google Drive not mounted (run the Drive cell first)")
        print("   2. Typo in the path")
        print("   3. Folder doesn't exist")

        # Try to help by showing parent folder contents
        parent = os.path.dirname(path)
        if os.path.exists(parent):
            print(f"\n📂 Contents of parent folder ({parent}):")
            for item in sorted(os.listdir(parent))[:15]:
                item_path = os.path.join(parent, item)
                if os.path.isdir(item_path):
                    print(f"   📁 {item}/")
                else:
                    print(f"   📄 {item}")
    elif not os.path.isdir(path):
        print(f"❌ Path is not a folder: {path}")
        print("   If this is a ZIP file, select 'Google Drive ZIP file' option.")
    else:
        input_folder = path
        print(f"✅ Using folder: {path}")
        list_folder_contents(input_folder)

# ============================================================
# OPTION 2: Google Drive ZIP file
# ============================================================
elif upload_method == "Google Drive ZIP file":
    path = drive_path.strip()

    if not path:
        print("❌ Please enter the path to your ZIP file.")
    elif not path.startswith('/content/drive/'):
        print("❌ Path must start with '/content/drive/MyDrive/'")
        print(f"   You entered: {path}")
    elif not os.path.exists(path):
        print(f"❌ ZIP file not found: {path}")
        print("\n💡 Make sure Google Drive is mounted and the path is correct.")
    elif not path.endswith('.zip'):
        print(f"❌ File is not a ZIP: {path}")
        print("   If this is a folder, select 'Google Drive path' option.")
    else:
        extract_dir = "/content/prediction_data"
        input_folder = extract_zip(path, extract_dir)
        print(f"✅ Extracted to: {input_folder}")
        list_folder_contents(input_folder)

# ============================================================
# OPTION 3: Upload ZIP from computer
# ============================================================
elif upload_method == "Upload ZIP file":
    print("📤 Please select your prediction data ZIP file...")
    print("   The ZIP should contain one or more job folders.\n")

    uploaded = files.upload()

    if uploaded:
        filename = list(uploaded.keys())[0]
        extract_dir = "/content/prediction_data"
        input_folder = extract_zip(filename, extract_dir)

        # Clean up uploaded ZIP
        if os.path.exists(filename):
            os.remove(filename)

        print(f"✅ Extracted to: {input_folder}")
        list_folder_contents(input_folder)
    else:
        print("❌ No file uploaded. Please run this cell again.")

# ============================================================
# Save input folder for next cell
# ============================================================
if input_folder:
    with open('/content/input_folder.txt', 'w') as f:
        f.write(input_folder)
    print("\n" + "="*60)
    print("✅ Data ready for analysis! Proceed to the next cell.")
    print("="*60)

In [ ]:
# @title **Run ipSAE Analysis** { display-mode: "form" }
# @markdown Execute the ipSAE_batch analysis on your selected data.

import subprocess
import sys
import os
from datetime import datetime

# Load input folder
try:
    with open('/content/input_folder.txt', 'r') as f:
        input_folder = f.read().strip()
except FileNotFoundError:
    print("❌ No input folder specified. Please run the 'Select Prediction Data' cell first.")
    raise SystemExit

if not os.path.exists(input_folder):
    print(f"❌ Input folder not found: {input_folder}")
    raise SystemExit

# Create output directory
output_dir = "/content/ipSAE_output"
if os.path.exists(output_dir):
    import shutil
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

print("="*60)
print("🔬 ipSAE BATCH ANALYSIS")
print("="*60)
print(f"📁 Input: {input_folder}")
print(f"📂 Output: {output_dir}")
print(f"🔧 Backend: {config['backend']}")
print(f"⏰ Started: {datetime.now().strftime('%H:%M:%S')}")
print("="*60 + "\n")

# Build command
cmd = [
    sys.executable, "-m", "ipsae_batch",
    input_folder,
    "--backend", config['backend'],
    "--output_dir", output_dir,
    "--pae_cutoff", str(config['pae_cutoff']),
    "--dist_cutoff", str(config['distance_cutoff']),
]

if config['generate_png']:
    cmd.append("--png")
if config['generate_pdf']:
    cmd.append("--pdf")
if config['per_contact_output']:
    cmd.append("--per_contact")
if config['per_residue_output']:
    cmd.append("--per_residue")

print(f"📋 Command: {' '.join(cmd)}\n")

# Run analysis with live output
start_time = datetime.now()
process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

for line in iter(process.stdout.readline, ''):
    print(line, end='')

process.wait()
elapsed = (datetime.now() - start_time).total_seconds()

print("\n" + "="*60)
if process.returncode == 0:
    print(f"✅ Analysis completed in {elapsed:.1f} seconds")

    # List output files
    print("\n📂 Output files:")
    for item in sorted(os.listdir(output_dir)):
        item_path = os.path.join(output_dir, item)
        if os.path.isfile(item_path):
            size_kb = os.path.getsize(item_path) / 1024
            print(f"   📄 {item} ({size_kb:.1f} KB)")

    # Count PNG files
    png_count = len([f for f in os.listdir(output_dir) if f.endswith('.png')])
    csv_count = len([f for f in os.listdir(output_dir) if f.endswith('.csv')])
    print(f"\n📊 Generated: {csv_count} CSV files, {png_count} PNG files")

    # Save output directory path
    with open('/content/output_dir.txt', 'w') as f:
        f.write(output_dir)

    print("\n✅ Proceed to the next cell to download results.")
else:
    print(f"❌ Analysis failed (exit code {process.returncode})")
    print("\nCommon issues:")
    print("   • Wrong backend selected for your data")
    print("   • Incorrect folder structure")
    print("   • Missing required files (PDB, PAE JSON)")
print("="*60)

In [ ]:
# @title **Download Results** { display-mode: "form" }
# @markdown Download your analysis results as a ZIP file or save to Google Drive.

# @markdown ---
save_to_drive = True # @param {type:"boolean"}
# @markdown Save results to Google Drive (if mounted)

download_zip = True # @param {type:"boolean"}
# @markdown Download results as ZIP file

# @markdown ---

import os
import shutil
from datetime import datetime
from google.colab import files

# Load output directory
try:
    with open('/content/output_dir.txt', 'r') as f:
        output_dir = f.read().strip()
except FileNotFoundError:
    print("❌ No output directory found. Please run the analysis first.")
    raise SystemExit

if not os.path.exists(output_dir):
    print(f"❌ Output directory not found: {output_dir}")
    raise SystemExit

print("="*60)
print("📦 EXPORT RESULTS")
print("="*60)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
archive_name = f"ipSAE_results_{timestamp}"

# Save to Google Drive
if save_to_drive:
    drive_results = "/content/drive/MyDrive/ipSAE_Results"
    if os.path.exists("/content/drive/MyDrive"):
        os.makedirs(drive_results, exist_ok=True)
        drive_output = os.path.join(drive_results, archive_name)

        print(f"\n💾 Copying to Google Drive...")
        shutil.copytree(output_dir, drive_output)
        print(f"✅ Saved to: {drive_output}")

        # List saved files
        file_count = len([f for f in os.listdir(drive_output) if os.path.isfile(os.path.join(drive_output, f))])
        print(f"   📁 {file_count} files saved")
    else:
        print("⚠️ Google Drive not mounted. Skipping Drive save.")

# Create downloadable ZIP
if download_zip:
    print(f"\n📦 Creating ZIP archive...")
    zip_path = f"/content/{archive_name}"
    shutil.make_archive(zip_path, 'zip', output_dir)
    zip_file = f"{zip_path}.zip"

    zip_size_mb = os.path.getsize(zip_file) / (1024 * 1024)
    print(f"✅ Created: {archive_name}.zip ({zip_size_mb:.2f} MB)")

    print("\n📥 Starting download...")
    files.download(zip_file)
    print("✅ Download initiated!")

print("\n" + "="*60)
print("🎉 Analysis Complete!")
print("="*60)
print("\n📊 Output includes:")
print("   • ipSAE_combined.csv - Aggregate scores for all jobs")
print("   • ipSAE_comparison.html - Interactive comparison plots")
if config.get('per_contact_output'):
    print("   • *_contacts.csv - Per-contact scores")
if config.get('generate_png'):
    print("   • *_matrix.png - PMC/Contact probability plots")
    print("   • *_ribbon.png - Circular ribbon diagrams")
print("\n📚 For more information, visit:")
print("   https://github.com/JKourelis/ipSAE_batch")

In [ ]:
# @title **Preview Results** (Optional) { display-mode: "form" }
# @markdown Display sample outputs inline.

# @markdown ---
show_summary_table = True # @param {type:"boolean"}
show_sample_plots = True # @param {type:"boolean"}
max_plots_to_show = 4 # @param {type:"integer"}
# @markdown ---

import os
import pandas as pd
from IPython.display import display, Image, HTML

# Load output directory
try:
    with open('/content/output_dir.txt', 'r') as f:
        output_dir = f.read().strip()
except FileNotFoundError:
    print("❌ No output directory found. Please run the analysis first.")
    raise SystemExit

print("="*60)
print("📊 RESULTS PREVIEW")
print("="*60)

# Show summary table
if show_summary_table:
    combined_csv = os.path.join(output_dir, "ipSAE_combined.csv")
    if os.path.exists(combined_csv):
        print("\n📋 Summary Table (ipSAE_combined.csv):")
        df = pd.read_csv(combined_csv)
        display(df.head(20))
        print(f"\n   Total rows: {len(df)}")
    else:
        print("⚠️ ipSAE_combined.csv not found")

# Show sample plots
if show_sample_plots:
    png_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.png')])

    if png_files:
        print(f"\n🖼️ Sample Plots ({min(len(png_files), max_plots_to_show)} of {len(png_files)}):")

        for png_file in png_files[:max_plots_to_show]:
            png_path = os.path.join(output_dir, png_file)
            print(f"\n   📄 {png_file}")
            display(Image(filename=png_path, width=600))
    else:
        print("⚠️ No PNG files found. Enable --png option to generate plots.")

# Show HTML comparison link
html_file = os.path.join(output_dir, "ipSAE_comparison.html")
if os.path.exists(html_file):
    print("\n📈 Interactive comparison available: ipSAE_comparison.html")
    print("   Download and open in browser for interactive plots.")